# Group API Project
Crime vs. Unemployment Rates

What, if any, relationship is there between national property crimes rates versus unemployment rates from 2010 to 2020?

Is there a noticeable difference between the years? Is there a potential correlation between crime and unemployment rates? Is there a stronger correlation in one subgroup of property crimes than the others?

In [46]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from pprint import pprint
import citipy
import json
from citipy import citipy
import seaborn as sns
import prettytable

#Federal Bureau of Investigation
#Crime Data Explorer
FBI_api_key="HXzuNSaaIO46x38N0iVVKpMDgrP6GG3mxBcacgRH"

#U.S. Bureau of Labor Statistics
BLS_api_key="489c914f30b34dac8ebc65feb25a15df"

## Performing the API Calls and Collecting the Data

In [47]:
#FBI JSON request
#base_url: https://api.usa.gov/crime/fbi/sapi/api/nibrs/{burglary}/offense/national/{count}

fbi_base_url = 'https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/larceny-theft-offenses/offense/national/count?limit=1&api_key='+ FBI_api_key
# This url will give us the number of offense with larceny-theft-offenses ^^^^^

# Changed to larceny from burglary (larceny is the broad definition)
fbi_larceny = requests.get(fbi_base_url)
fbi_json_data_larc = json.loads(fbi_larceny.text)
print(fbi_json_data_larc)



# I think we are going to need population data as well so that we can create a percent of larceny offensers to total
# population.  That way we can effectively compare it to the unemployment percentage.

{'results': [{'incident_count': 5625, 'offense_count': 5625, 'data_year': 2015}, {'incident_count': 2413, 'offense_count': 2413, 'data_year': 1992}, {'incident_count': 204092, 'offense_count': 204092, 'data_year': 2005}, {'incident_count': 3052, 'offense_count': 3052, 'data_year': 2000}, {'incident_count': 6209, 'offense_count': 6209, 'data_year': 2007}, {'incident_count': 82094, 'offense_count': 82094, 'data_year': 1997}, {'incident_count': 903, 'offense_count': 903, 'data_year': 1996}, {'incident_count': 3910, 'offense_count': 3910, 'data_year': 2014}, {'incident_count': 162511, 'offense_count': 162511, 'data_year': 1993}, {'incident_count': 436104, 'offense_count': 436104, 'data_year': 2017}, {'incident_count': 391497, 'offense_count': 391497, 'data_year': 2012}, {'incident_count': 3791, 'offense_count': 3791, 'data_year': 2015}, {'incident_count': 1438, 'offense_count': 1438, 'data_year': 1992}, {'incident_count': 90774, 'offense_count': 90774, 'data_year': 1998}, {'incident_count'

In [48]:
#organizing json for better analysis
fbi_json = fbi_larceny.json()

fbi_json


{'results': [{'incident_count': 5625,
   'offense_count': 5625,
   'data_year': 2015},
  {'incident_count': 2413, 'offense_count': 2413, 'data_year': 1992},
  {'incident_count': 204092, 'offense_count': 204092, 'data_year': 2005},
  {'incident_count': 3052, 'offense_count': 3052, 'data_year': 2000},
  {'incident_count': 6209, 'offense_count': 6209, 'data_year': 2007},
  {'incident_count': 82094, 'offense_count': 82094, 'data_year': 1997},
  {'incident_count': 903, 'offense_count': 903, 'data_year': 1996},
  {'incident_count': 3910, 'offense_count': 3910, 'data_year': 2014},
  {'incident_count': 162511, 'offense_count': 162511, 'data_year': 1993},
  {'incident_count': 436104, 'offense_count': 436104, 'data_year': 2017},
  {'incident_count': 391497, 'offense_count': 391497, 'data_year': 2012},
  {'incident_count': 3791, 'offense_count': 3791, 'data_year': 2015},
  {'incident_count': 1438, 'offense_count': 1438, 'data_year': 1992},
  {'incident_count': 90774, 'offense_count': 90774, 'data

In [49]:
#creating dataframe, sorting by year, then filtering for 2011-2020
fbi_df=pd.DataFrame(fbi_json['results'])
fbi_sorted=fbi_df.sort_values(by='data_year')
fbi_yr_filtered = fbi_sorted.loc[fbi_sorted['data_year'] >= 2011]
pd.DataFrame(fbi_yr_filtered)
# Why are there multiple entries for each year???

,incident_count,offense_count,data_year
89,290203,290203,2011
118,6020,6020,2011
93,387586,387586,2011
225,688624,688624,2011
149,6072,6072,2011
...,...,...,...
92,5009,5009,2020
136,502563,502563,2020
51,914347,914347,2020
233,164192,164192,2020


In [50]:
raw_fbi_data_larc = []
raw_fbi_data_larc.append(fbi_json_data_larc)
fbi_data_unformed_larc = pd.json_normalize(raw_fbi_data_larc, sep='_')
#fbi_data_unformed_larc = fbi_data_unformed_larc.set_index("ui_type")
fbi_data_unformed_larc.to_csv('fbi_data_unformed_larc_csv.csv')

In [58]:
#BLS JSON request
base_url="https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey="+BLS_api_key+"&catalog=false&startyear=2011&endyear=2020&calculations=true&annualaverage=true&aspects=true"

# changed start year to 2011 so we can get 2020 data in the API call
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2020"})
bls_unemp = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
bls_json_data_unemp = json.loads(bls_unemp.text)
for series in bls_json_data_unemp['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()

In [52]:
print(bls_json_data_unemp)

{'status': 'REQUEST_SUCCEEDED', 'responseTime': 279, 'message': [], 'Results': {'series': [{'seriesID': 'CUUR0000SA0', 'data': [{'year': '2020', 'period': 'M12', 'periodName': 'December', 'value': '260.474', 'footnotes': [{}]}, {'year': '2020', 'period': 'M11', 'periodName': 'November', 'value': '260.229', 'footnotes': [{}]}, {'year': '2020', 'period': 'M10', 'periodName': 'October', 'value': '260.388', 'footnotes': [{}]}, {'year': '2020', 'period': 'M09', 'periodName': 'September', 'value': '260.280', 'footnotes': [{}]}, {'year': '2020', 'period': 'M08', 'periodName': 'August', 'value': '259.918', 'footnotes': [{}]}, {'year': '2020', 'period': 'M07', 'periodName': 'July', 'value': '259.101', 'footnotes': [{}]}, {'year': '2020', 'period': 'M06', 'periodName': 'June', 'value': '257.797', 'footnotes': [{}]}, {'year': '2020', 'period': 'M05', 'periodName': 'May', 'value': '256.394', 'footnotes': [{}]}, {'year': '2020', 'period': 'M04', 'periodName': 'April', 'value': '256.389', 'footnotes

In [59]:
#organizing json data for easier analysis
bls_json = bls_unemp.json()

bls_json


{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 298,
 'message': [],
 'Results': {'series': [{'seriesID': 'CUUR0000SA0',
    'data': [{'year': '2020',
      'period': 'M12',
      'periodName': 'December',
      'value': '260.474',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M11',
      'periodName': 'November',
      'value': '260.229',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M10',
      'periodName': 'October',
      'value': '260.388',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M09',
      'periodName': 'September',
      'value': '260.280',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M08',
      'periodName': 'August',
      'value': '259.918',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M07',
      'periodName': 'July',
      'value': '259.101',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M06',
      'periodName': 'June',
      'value': '257.797',
   

In [56]:
#creating dataframe
bls_df=pd.DataFrame(bls_json['Results']['series'][0]['data'])
#sort by year
bls_sorted=bls_df.sort_values(by=['year','period'])
pd.DataFrame(bls_sorted)


,year,period,periodName,value,footnotes
119,2011,M01,January,220.223,[{}]
118,2011,M02,February,221.309,[{}]
117,2011,M03,March,223.467,[{}]
116,2011,M04,April,224.906,[{}]
115,2011,M05,May,225.964,[{}]
...,...,...,...,...,...
4,2020,M08,August,259.918,[{}]
3,2020,M09,September,260.280,[{}]
2,2020,M10,October,260.388,[{}]
1,2020,M11,November,260.229,[{}]


In [60]:
annual_avg=bls_sorted.groupby('year').agg({'value'):'mean'})
print(annual_avg)

#line plot for year v. value -- it sees these values as strings though, and won't let me convert to integers

#bls_sorted.plot(x='year', y=float('value'), kind='line')
#plt.show()

ValueError: could not convert string to float: 'value'

#code from Week 5 to help me remember the steps
#cleaning the data
cities_weather_unformed = pd.json_normalize(raw_weather, sep='_')
cities_weather_unformed = cities_weather_unformed.drop(cities_weather_unformed[cities_weather_unformed.message == "city not found"].index)

cities_weather_unformed = cities_weather_unformed.set_index("name")
cities_weather_unformed.to_csv('cities_weather_unformed_csv.csv')

#building a focused dataframe
cities_weather = cities_weather_unformed.filter(["sys_country", "coord_lon", "coord_lat", "main_pressure", "main_temp_max", "wind_speed", "main_humidity", "clouds_all"], axis=1)
cities_weather = pd.DataFrame(cities_weather)

#export to a csv for confirmation
cities_weather.to_csv('cities_weather_csv.csv')

#print a sample to show it works
cities_weather.head(75)

#code from Week 5 to help me remember the steps

raw_weather = []

base_url="http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key

try:
    for city in cities:
        req = requests.get(base_url + f'&q={city}')
        json_data = req.json()
        raw_weather.append(json_data)
        print("Processing "+ city)

except requests.exceptions.RequestException as e:
        print("Processing "+ city + "failed.")
